In [1]:
# 在python中，glob模块是用来查找匹配的文件的
#     在查找的条件中，需要用到Unix shell中的匹配规则：
#        *    :   匹配所所有
#        ?    :   匹配一个字符
#        *.*  :   匹配如：[hello.txt,cat.xls,xxx234s.doc]
#        ?.*  :   匹配如：[1.txt,h.py]
#        ?.gif:   匹配如：[x.gif,2.gif]
#     如果没有匹配的，glob.glob(path)将返回一个空的list:[]


# import glob  
# def get_all():  
#     '''''获取目录[F:\\wfpdm\\My_Proc_Data_ZXTZ]下面所有的文件'''  
#     return glob.glob('F:\\wfpdm\\My_Proc_Data_ZXTZ\\*.*')  
# def get_my_file():  
#     '''''获取目录[F:\\wfpdm\\My_Proc_Data_ZXTZ]下面文件名为4个字符的文件'''  
#     return glob.glob('F:\\wfpdm\\My_Proc_Data_ZXTZ\\????.txt')
# def get_batch_file():  
#     '''''获取目录[F:\\wfpdm\\My_Proc_Data_ZXTZ]下面扩展名为\'.txt\'的文件'''  
#     return glob.glob('F:\\wfpdm\\My_Proc_Data_ZXTZ\\*.txt')  
# def main():  
#     print('获取目录[F:\\wfpdm\\My_Proc_Data_ZXTZ]下面所有的文件：')
#     tem_files = get_all()  
#     print(tem_files)  
#     print('获取目录[F:\\wfpdm\\My_Proc_Data_ZXTZ]下面文件名为4个字符的文件：')  
#     tem_files = get_my_file()  
#     print(tem_files)  
#     print('获取目录[F:\\wfpdm\\My_Proc_Data_ZXTZ]下面扩展名为\'.txt\'的文件：')  
#     tem_files = get_batch_file()  
#     print(tem_files)  
# if __name__ == '__main__':  
#     main()  

In [132]:
from os.path import exists, isdir, basename, join, splitext,split
from glob import glob
import pandas as pd
EXTENSIONS = ['.csv']
input_path = "/home/mi/workspace/data/NationalData"
output_path = input_path+"/result"
def get_categories(dir_path):
    '''得到所有分类，文件夹名称'''
    cat_paths = [files
                 for files in glob(dir_path + "/*")
                  if isdir(files)]
    cat_paths.sort()
    cats = [basename(cat_path) for cat_path in cat_paths if basename(cat_path)!="result" ]
    return cats

def get_files(path, extensions=EXTENSIONS):
    '''返回分类路径path下的所有视频文件名,list'''
    all_files = []
    all_files.extend([join(path, basename(fname)) for fname in glob(path + "/*.*") 
                      if splitext(fname)[1] in extensions])
    return all_files

def get_data(files):
    df_list = []
    df_list = [pd.read_csv(file,skiprows=2,skipfooter=1,skipinitialspace=True,encoding="gbk")
               for file in files if exists(file)]
    df_list = [df.dropna(thresh=len(df)-1) for df in df_list]
    return pd.concat(df_list,ignore_index=True)
    
def merge_data(path):
    files = [file for file in glob(path+"/*")]
    df_list = [pd.read_csv(file,sep='\t') for file in files if basename(file)!="result"]
    res_df = pd.concat(df_list,ignore_index=True)
    cols = list(res_df.columns)
    cols.remove("category")
    cols.remove("index")
    res_df.loc[:,["category","index"]+cols].to_parquet(join(path,"result"))
    
if __name__=="__main__":
    cats = get_categories(input_path)
    for dir_name in cats:
        all_files = get_files(join(input_path,dir_name))
        df = get_data(all_files)
        cols = df.columns
        cols = [i.replace("年","").replace("月","") for i in cols]
        cols = [i[:4]+"0"+i[4:] if i!="指标" and int(i[4:])<10 else i for i in cols]
        df.columns = cols
        cols.remove("指标")
        cols.sort()
        df["category"] = dir_name
        df["index"] = df["指标"]
        df.drop(columns="指标",inplace=True)
        df = df.loc[:,["category","index"]+cols]
        df.to_csv(join(output_path,dir_name)+".csv",sep='\t',index=False)   
    merge_data(output_path)

/home/mi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/home/mi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/mi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [140]:
df = pd.read_parquet("/home/mi/workspace/data/NationalData/result/result")

In [141]:
len(df)

2254

In [131]:
import os
path = "/home/mi/workspace/data/NationalData/工业"
files = glob(path+"/*.*")
for file in files:
    df = pd.read_csv(file,skiprows=2,skipfooter=1,skipinitialspace=True,encoding="gbk")
    df = df.dropna(thresh=len(df)-1)
    df["index"] = df["指标"]
    df.drop(columns="指标",inplace=True)
#     print(df.columns)
    val = df["index"].isin(["其他采矿业出口交货值当期值(亿元)"]).any()
    if val:
        print("#######"+os.path.basename(file))
        
#     if len(df.columns)==19:
#         print(os.path.basename(file))

/home/mi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """


#######月度数据 (12).csv
#######月度数据 (54).csv


In [137]:
df = pd.read_csv("/home/mi/workspace/data/NationalData/result/工业.csv",sep='\t')

In [139]:
df["index"].isin(["其他采矿业出口交货值当期值(亿元)"]).any()

False